<a href="https://colab.research.google.com/github/ED19841002/testrepo/blob/main/Space_X_Falcon9_EDA_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sqlalchemy
!pip install ipython-sql
!pip install ipython-sql prettytable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:00


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas

In [5]:
%sql sqlite:///my_data1.db

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql('SPACEXTBL', con, if_exists='replace', index=False, method="multi")

101

In [8]:
%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [9]:
%sql CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE DATE IS NOT NULL

 * sqlite:///my_data1.db
Done.


[]

In [13]:
#Display the names of the unique launch sites in the space mission
%sql SELECT DISTINCT Launch_Site FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [16]:
#Display 5 records where launch sites begin with the string 'CCA'
%sql SELECT *\
FROM SPACEXTABLE\
WHERE Launch_Site LIKE 'CCA%'\
LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [18]:
#Display the total payload mass carried by boosters launched by NASA (CRS)
%sql SELECT SUM(PAYLOAD_MASS__KG_) AS TOTAL_PAYLOAD_MASS\
FROM SPACEXTABLE\
WHERE Customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


TOTAL_PAYLOAD_MASS
45596


In [21]:
#Display average payload mass carried by booster version F9 v1.1
%sql SELECT ROUND((AVG(PAYLOAD_MASS__KG_)), 0) AS AVG_PAYLOAD_MASS\
FROM SPACEXTABLE\
WHERE Booster_Version LIKE 'F9 v1.1%';

 * sqlite:///my_data1.db
Done.


AVG_PAYLOAD_MASS
2535.0


In [25]:
#List the date when the first succesful landing outcome in ground pad was acheived.
%sql SELECT MIN(Date) AS FIRST_GROUND_LAND\
FROM SPACEXTABLE\
WHERE Landing_Outcome = 'Success (ground pad)';


 * sqlite:///my_data1.db
Done.


FIRST_GROUND_LAND
2015-12-22


In [26]:
#List the names of the boosters which have success in drone ship
# and have payload mass greater than 4000 but less than 6000
%sql SELECT Booster_Version \
FROM SPACEXTABLE\
WHERE Landing_Outcome = 'Success (drone ship)'\
 AND PAYLOAD_MASS__KG_ > 4000 AND PAYLOAD_MASS__KG_ < 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [36]:
#List the total number of successful and failure mission outcomes
%sql SELECT Mission_Outcome, COUNT(*) AS TOTAL_COUNT\
FROM SPACEXTABLE\
GROUP BY Mission_Outcome;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,TOTAL_COUNT
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


In [41]:
%sql SELECT\
(SELECT COUNT(*) FROM SPACEXTABLE WHERE Mission_Outcome LIKE 'Success%') AS Success,\
(SELECT COUNT(*) FROM SPACEXTABLE WHERE Mission_Outcome LIKE 'Failure%') AS Failure;


 * sqlite:///my_data1.db
Done.


Success,Failure
100,1


In [48]:
#List the names of the booster_versions which have carried the maximum payload mass. Use a subquery
%sql SELECT Booster_Version, PAYLOAD_MASS__KG_ AS Max_Mass\
FROM SPACEXTABLE\
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE);


 * sqlite:///my_data1.db
Done.


Booster_Version,Max_Mass
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


In [58]:
#List the records which will display the month names,
# failure landing_outcomes in drone ship ,booster versions,
# launch_site for the months in year 2015. (SQLlite version)
%sql SELECT  \
  CASE substr(Date, 6, 2)\
        WHEN '01' THEN 'January'\
        WHEN '02' THEN 'February'\
        WHEN '03' THEN 'March'\
        WHEN '04' THEN 'April'\
        WHEN '05' THEN 'May'\
        WHEN '06' THEN 'June'\
        WHEN '07' THEN 'July'\
        WHEN '08' THEN 'August'\
        WHEN '09' THEN 'September'\
        WHEN '10' THEN 'October'\
        WHEN '11' THEN 'November'\
        WHEN '12' THEN 'December'\
    END AS Month_Name, Landing_Outcome, Booster_Version, Launch_Site\
FROM SPACEXTABLE\
WHERE substr(Date,0,5)='2015' AND Landing_Outcome LIKE '%Failure (drone ship)%';

 * sqlite:///my_data1.db
Done.


Month_Name,Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [69]:
#Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad))
#between the date 2010-06-04 and 2017-03-20, in descending order.
%sql SELECT Landing_Outcome, COUNT(Landing_Outcome) AS No_Outcomes\
FROM SPACEXTABLE\
WHERE DATE BETWEEN '2010-06-04' AND '2017-03-20'\
GROUP BY Landing_Outcome ORDER BY COUNT(Landing_Outcome) DESC ;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,No_Outcomes
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
